In [64]:
# Imports

import pandas as pd
import numpy as np
import re
from Scripts.normalize_data import getTrueLabelsDataframe, getCleanDataframe, getCleanAiDataframe, apply_normalizations
from Scripts.functions import reder, bench_it
from pathlib import Path
import os


In [65]:
df_ground_truth = getTrueLabelsDataframe()

## Loading Data to analyse


In [66]:
#loading dataframes
bandit = getCleanDataframe('C:/Users/levim/projetos/results_mk2/formatted_bandit.json').rename(columns={'labels': 'bandit'})
semgrep = getCleanDataframe('C:/Users/levim/projetos/results_mk2/formatted_semgrep.json').rename(columns={'labels': 'semgrep'})
ai_bandit = getCleanAiDataframe('C:/Users/levim/projetos/results_mk2/ai_results_bandit.json').rename(columns={'ai_predictions': 'ai_bandit'})
ai_semgrep = getCleanAiDataframe('C:/Users/levim/projetos/results_mk2/ai_results_semgrep.json').rename(columns={'ai_predictions': 'ai_semgrep'})


#Merging all of them into a single one containing the ground truths
labels = (
    df_ground_truth
    .merge(bandit[['filename', 'bandit']], on='filename', how='left')
    .merge(semgrep[['filename', 'semgrep']], on='filename', how='left')
    .merge(ai_bandit[['filename', 'ai_bandit']], on='filename', how='left')
    .merge(ai_semgrep[['filename', 'ai_semgrep']], on='filename', how='left')
)

#Applying normalizations on the columns
labels = apply_normalizations(labels, 'bandit')
labels = apply_normalizations(labels, 'semgrep')
labels = apply_normalizations(labels, 'ai_bandit')
labels = apply_normalizations(labels, 'ai_semgrep')

def remove_spaces(text):
    return re.sub(r'(\d) +\(', r'\1(', text)

labels['true_label'] = labels['true_label'].apply(remove_spaces)

labels

,filename,true_label,bandit,semgrep,ai_bandit,ai_semgrep
0,command_injection.py,"16(CWE-78),18(CWE-78),27(CWE-78),35(CWE-489),4...","18(CWE-78),2(CWE-78),27(CWE-78),35(CWE-94)","18(CWE-78),35(CWE-489)",20(CWE-78),"22(CWE-78),37(CWE-489)"
1,path_traversal.py,"11(CWE-22),15(CWE-489),4(CWE-15)",15(CWE-94),15(CWE-489),nan,16(CWE-489)
2,path_traversal_sanitised.py,"12(CWE-22),15(CWE-489),4(CWE-15)",15(CWE-94),15(CWE-489),nan,15(CWE-489)
3,path_traversal_sanitised_2.py,"12(CWE-22),15(CWE-489),4(CWE-15)",15(CWE-94),15(CWE-489),nan,16(CWE-489)
4,simple_vulnerability.py,"2(CWE-78),4(CWE-78)",4(CWE-78),4(CWE-95),nan,nan
...,...,...,...,...,...,...
151,CWE-841_mitre_1.py,"10(CWE-841),11(CWE-841),12(CWE-841),13(CWE-841...",nan,nan,nan,nan
152,CWE-918_codeql_1.py,"11(CWE-918),12(CWE-918),13(CWE-918),14(CWE-918)",13(CWE-400),12(CWE-918),13(CWE-400),13(CWE-918)
153,CWE-918_codeql_2.py,"11(CWE-918),12(CWE-918),13(CWE-918),14(CWE-918)",13(CWE-400),11(CWE-918),11(CWE-400),13(CWE-918)
154,CWE-941_mitre_1.py,"10(CWE-941),11(CWE-941),12(CWE-941),13(CWE-941...",nan,nan,nan,nan


## Evaluating the results




In [67]:
#Generating list of predictions
banditPreds = reder(labels, 'bandit')
semgrepPreds = reder(labels, 'semgrep')
aiBanditPreds = reder(labels, 'ai_bandit')
aiSemgrepPreds = reder(labels, 'ai_semgrep')

#Generating list of expected values
exploded_labels = labels['true_label'].str.split(',').explode()
dist_labels = (exploded_labels.index.astype(str) + '-' + exploded_labels).values

teste = {
    'bandit': banditPreds,
    'semgrep': semgrepPreds,
    'ai_bandit': aiBanditPreds,
    'ai_semgrep': aiSemgrepPreds
}


In [68]:
rows = []

for tool, preds in teste.items():
    precision, recall, f1 = bench_it(dist_labels, preds)
    
    rows.append({
        'Tool': tool,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

report_df = pd.DataFrame(rows)

report_df

,Tool,Precision,Recall,F1 Score
0,bandit,0.285714,0.057803,0.096154
1,semgrep,0.677778,0.117534,0.200328
2,ai_bandit,0.328358,0.042389,0.075085
3,ai_semgrep,0.389610,0.057803,0.100671
